In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/lyrics_original/lyric_word_parsed_by_line.csv"
words = pd.read_csv(url, encoding="utf-8").iloc[:, 1:]
words


,genre,year,song_id,title,line,word,class,tag_orig
0,성인가요,1964,4083218,내일또 만납시다,0,하루,명사,NNG
1,성인가요,1964,4083218,내일또 만납시다,0,일,명사,NNG
2,성인가요,1964,4083218,내일또 만납시다,0,끝내,용언,VV
3,성인가요,1964,4083218,내일또 만납시다,0,돌아가,용언,VV
4,성인가요,1964,4083218,내일또 만납시다,1,거리,명사,NNG
...,...,...,...,...,...,...,...,...
342247,"발라드, 국내드라마",2020,32323639,어떤 날엔,39,흐르,용언,VV
342248,"발라드, 국내드라마",2020,32323639,어떤 날엔,39,눈물,명사,NNG
342249,"발라드, 국내드라마",2020,32323639,어떤 날엔,40,닦,용언,VV
342250,"발라드, 국내드라마",2020,32323639,어떤 날엔,40,오,용언,VV


In [2]:
def drop_non_cooccured_lines(line):
    if (
        len(line) < 1
        or "명사" not in [each[1] for each in line["class"].items()]
        or "용언" not in [each[1] for each in line["class"].items()]
    ):
        return None
    return line


processed_words = (
    words.groupby(["song_id", "line"])
    .apply(drop_non_cooccured_lines)
    .reset_index(drop=True)
    .sort_values(by=["year", "title", "line"])
)
processed_words


,genre,year,song_id,title,line,word,class,tag_orig
190467,성인가요,1964,4083218,내일또 만납시다,0,하루,명사,NNG
190468,성인가요,1964,4083218,내일또 만납시다,0,일,명사,NNG
190469,성인가요,1964,4083218,내일또 만납시다,0,끝내,용언,VV
190470,성인가요,1964,4083218,내일또 만납시다,0,돌아가,용언,VV
190471,성인가요,1964,4083218,내일또 만납시다,2,하늘,명사,NNG
...,...,...,...,...,...,...,...,...
241006,발라드,2020,32998018,힘든 건 사랑이 아니다,15,그늘,명사,NNG
241007,발라드,2020,32998018,힘든 건 사랑이 아니다,15,가리,용언,VV
241008,발라드,2020,32998018,힘든 건 사랑이 아니다,15,사랑,명사,NNG
241009,발라드,2020,32998018,힘든 건 사랑이 아니다,15,받,용언,VV


In [3]:
# 명사 - 용언 교차 공기 데이터 생성
def create_cooccured_data(line):
    nouns = line[line["class"] == "명사"]
    predicates = line[line["class"] == "용언"]
    line_df = pd.DataFrame(
        {
            "genre": [],
            "year": [],
            "song_id": [],
            "title": [],
            "line": [],
            "noun": [],
            "predicate": [],
        }
    )
    genre = line["genre"].iloc[0]
    year = line["year"].iloc[0]
    song_id = line["song_id"].iloc[0]
    title = line["title"].iloc[0]
    line_ = line["line"].iloc[0]
    print(year, title, line_, sep=" ")
    for _, noun in nouns.iterrows():
        for _, predicate in predicates.iterrows():
            line_df = pd.concat(
                (
                    line_df,
                    pd.DataFrame(
                        {
                            "genre": [genre],
                            "year": [year],
                            "song_id": [song_id],
                            "title": [title],
                            "line": [line_],
                            "noun": [noun.word],
                            "predicate": [predicate.word],
                        }
                    ),
                )
            )
    return line_df


cooccured_words = (
    processed_words.groupby(["song_id", "line"])
    .apply(create_cooccured_data)
    .reset_index(drop=True)
    .sort_values(by=["year", "title", "line"])
)
cooccured_words


2001 가져가 2
2001 가져가 3
2001 가져가 7
2001 가져가 11
2001 가져가 13
2001 가져가 14
2001 가져가 21
2001 가져가 23
2001 가져가 25
2001 가져가 27
2001 가져가 28
2001 가져가 29
2001 가져가 34
2001 가져가 35
2001 가져가 38
2001 가져가 45
2001 가져가 47
2001 가져가 53
2001 가져가 55
2000 너를 위해 1
2000 너를 위해 3
2000 너를 위해 6
2000 너를 위해 8
2000 너를 위해 9
2000 너를 위해 10
2000 너를 위해 12
2000 너를 위해 14
2000 너를 위해 17
2000 너를 위해 18
2000 너를 위해 19
2000 너를 위해 21
2000 너를 위해 23
2000 와인(迗人) 1
2000 와인(迗人) 2
2000 와인(迗人) 3
2000 와인(迗人) 4
2000 와인(迗人) 5
2000 와인(迗人) 7
2000 와인(迗人) 8
2000 와인(迗人) 10
2000 와인(迗人) 11
2000 와인(迗人) 12
2000 와인(迗人) 13
2000 와인(迗人) 15
2000 와인(迗人) 16
2000 와인(迗人) 18
2000 와인(迗人) 20
2000 와인(迗人) 21
2000 와인(迗人) 22
2000 와인(迗人) 23
2000 와인(迗人) 25
2000 와인(迗人) 26
2000 와인(迗人) 28
2000 와인(迗人) 29
2000 와인(迗人) 30
2000 와인(迗人) 31
2000 와인(迗人) 32
2000 와인(迗人) 34
2000 와인(迗人) 35
2000 와인(迗人) 36
2001 그런 일은 3
2001 그런 일은 11
2001 그런 일은 12
2001 그런 일은 13
2001 그런 일은 14
2001 그런 일은 18
2001 그런 일은 25
2001 그런 일은 26
2001 그런 일은 28
2001 그런 일은 29
2001 그런 일은 32
2000 831 8 0
2000 831 8 3
2000 8

,genre,year,song_id,title,line,noun,predicate
145487,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루,끝내
145488,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루,돌아가
145489,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일,끝내
145490,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일,돌아가
145491,성인가요,1964.0,4083218.0,내일또 만납시다,2.0,하늘,반짝이
...,...,...,...,...,...,...,...
182874,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,그늘,받
182875,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑,가리
182876,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑,받
182877,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,미안,가리


In [4]:
# 명사, 용언 노드 태그 추가
cooccured_words["noun"] = cooccured_words["noun"].astype(str) + "(N)"
cooccured_words["predicate"] = cooccured_words["predicate"].astype(str) + "(P)"
cooccured_words


,genre,year,song_id,title,line,noun,predicate
145487,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루(N),끝내(P)
145488,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루(N),돌아가(P)
145489,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일(N),끝내(P)
145490,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일(N),돌아가(P)
145491,성인가요,1964.0,4083218.0,내일또 만납시다,2.0,하늘(N),반짝이(P)
...,...,...,...,...,...,...,...
182874,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,그늘(N),받(P)
182875,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑(N),가리(P)
182876,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑(N),받(P)
182877,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,미안(N),가리(P)


In [5]:
cooccured_words.reset_index(drop=True).to_csv(
    "../data/lyrics_cooccured/lyric_cooccured_by_line.csv", encoding="utf-8-sig"
)
